In [23]:
import pandas as pd
from openai import OpenAI
from sklearn.metrics import accuracy_score

In [ ]:
client = OpenAI(api_key = 'sk-proj')

In [25]:
df = pd.read_csv('framingham.csv')

In [26]:
examples = df.iloc[:10]
example_blocks = []

for i in range(10):
    row = examples.iloc[i]
    features = row.drop('TenYearCHD')
    label = row['TenYearCHD']
    example_blocks.append(f"{features.to_string(index=True)}\nAnswer: {label}")

few_shot_examples = "\n\n".join(example_blocks)
print(few_shot_examples)

male                 1.00
age                 39.00
education            4.00
currentSmoker        0.00
cigsPerDay           0.00
BPMeds               0.00
prevalentStroke      0.00
prevalentHyp         0.00
diabetes             0.00
totChol            195.00
sysBP              106.00
diaBP               70.00
BMI                 26.97
heartRate           80.00
glucose             77.00
Answer: 0.0

male                 0.00
age                 46.00
education            2.00
currentSmoker        0.00
cigsPerDay           0.00
BPMeds               0.00
prevalentStroke      0.00
prevalentHyp         0.00
diabetes             0.00
totChol            250.00
sysBP              121.00
diaBP               81.00
BMI                 28.73
heartRate           95.00
glucose             76.00
Answer: 0.0

male                 1.00
age                 48.00
education            1.00
currentSmoker        1.00
cigsPerDay          20.00
BPMeds               0.00
prevalentStroke      0.00
prevalentHyp

In [27]:
test_df = df.iloc[10:33]
features_sample = test_df.drop(columns=['TenYearCHD'])
target_sample = test_df['TenYearCHD']

In [28]:
def llm_predict(row):
    row_string = row.to_string(index=True)

    prompt = f"""You are a cardiovascular disease risk prediction expert.

Based on structured patient data, predict whether the person is at risk of developing coronary heart disease within 10 years.
Return only 1 (at risk) or 0 (not at risk).

Here are a few examples:
{few_shot_examples}

Now predict for the following case:
{row_string}
Answer:"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # ya da "gpt-4"
        messages=[
            {"role": "system", "content": "You are a medical expert specialized in cardiovascular risk prediction."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    output = response.choices[0].message.content.strip()
    return int('1' in output)

In [29]:
preds = features_sample.apply(llm_predict, axis=1)

In [30]:
acc = accuracy_score(target_sample, preds)
print(f"Accuracy: {acc:.2f}")

Accuracy: 0.22
